A small notebook explaining the pitfalls of algorithmic compexities and the difference of complexity and actual runtime.

First, let's define a few mockup functions whose performance we want to analyze.

In [1]:
# mockup for a function with constant runtime:
# it simply performs a multiplication on the input parameters
# this function has constant runtime,
# basically just the costs for one multiplication
def foo(i,j):
    return i*j

# mockup for a function with linear runtime:
# this function calls function <func> n times
def loop(n, func):
    sum = 0
    for i in range(n):
        sum += func(i,i)
    return sum

# mockup for a function with quadratic runtime:
# this function calls function <func> n*n times
def nestedLoop(n, func):
    sum = 0
    for i in range(n):
        for j in range(n):
            sum += func(i,j)
    return sum

# mockup for a function with n*log n runtime:
# this function calls function <func> n*ceil(log_2 n) times
import math
def loopLog(n, func):
    sum = 0
    _log = math.ceil(math.log2(n))
    for i in range(n):
        for j in range(_log):
            sum += func(i,j)
    return sum

In [2]:
# list files in current directory greater than 1MB
# we do this to simulate I/O operations to play with:
import os
files = [(f,os.stat(f).st_size) for f in os.listdir('.') if os.path.isfile(f) and os.stat(f).st_size >1024*1024]
print(files)

# a mockup function performing a pseudo-random read to simulate I/O:
def foo_IO(i,j):
    # pick a file pseudo-randomly from the pre-computed list of files:
    pseudo_random = i*i % len(files)
    with open(files[pseudo_random][0], 'rb') as f:
        # seek to random offset in that file:
        f.seek(files[pseudo_random][1]//2)
        # read 4096 bytes and return the last byte read:
        first_bytes = f.read(4096)
        # return the bytes read:
        return first_bytes[min(4095,len(first_bytes)-1)]

[('09 Zusammenfassung.pdf', 13174793), ('x9 Datenjournalismus (Teil 1).pdf', 16935169), ('00 Jupyter.ipynb', 3889160), ('01 IMDb (Teil 1).pdf', 12137353), ('x10 Datenjournalismus (Teil 2).pdf', 8853176), ('05 Anfrageoptimierung (Teil 1).pdf', 7176671), ('07 Handel, Banken, Ticketsystem (Teil 1).pdf', 33598178), ('Welcome.pdf', 6486308), ('02 IMDb (Teil 2).pdf', 4143455)]


Let's run performance measurements on these mockup functions:

In [3]:
import time
from tqdm import tqdm
# combination of methods to benchmark:
# syntax: (outer-function, inner-function)
methods=[ (loop,foo), (loop, foo_IO), (loopLog, foo), (nestedLoop, foo)]
results = []
for m in tqdm(methods):
    # vary n:
    for n in range(100, 10000, 500):
        start = time.time()
        # call the function combination:
        m[0](n, m[1])
        elapsedTime = time.time() - start
        # append measured times to measurement data:
        results.append((m[0].__name__, m[1].__name__, n, elapsedTime))

100%|██████████| 4/4 [01:09<00:00, 17.28s/it]


Let's display the performance results visually:

In [4]:
import altair as alt
alt.renderers.enable('default')
import pandas as pd

# display all methods:
# convert the <results> array to a pandas dataframe (a table-like structure)
resultsPD = pd.DataFrame(results, columns=['method', 'foo', 'n', 'time'])

# map the columns of resultsPD to x, y, and color
alt.Chart(resultsPD).transform_calculate(
    cat="datum.method + ' using ' + datum.foo"
).mark_line().encode(
    x='n',
    y=alt.Y('time', title='Time [sec]'),
    color=alt.Y('cat:N', title='Algorithm'),
).interactive()

alt.Chart(...)

In [5]:
# display the data as a table:
resultsPD

,method,foo,n,time
0,loop,foo,100,0.000011
1,loop,foo,600,0.000054
2,loop,foo,1100,0.000099
3,loop,foo,1600,0.000147
4,loop,foo,2100,0.000255
...,...,...,...,...
75,nestedLoop,foo,7600,4.570703
76,nestedLoop,foo,8100,5.379767
77,nestedLoop,foo,8600,5.935021
78,nestedLoop,foo,9100,6.633059


feel free to ignore the following...

In [6]:
# helper to be able to tun SQL against pandas data frames:
import pandasql as pdsql
pysql = lambda q: pdsql.sqldf(q, globals())
import sqlparse

def queryFrame(q, _display=False):
    result = pysql(q)
    if _display:
        display(result)
    return result

In [7]:
# calculate performance difference of different methods as a facor:
improvementFactor = queryFrame("""
WITH justfoo AS (
    SELECT *
    FROM resultsPD
    WHERE method='loop' AND foo='foo'),
justRead AS (
    SELECT *
    FROM resultsPD
    WHERE method='loop' AND foo='foo_IO')

SELECT L.n, R.time/L.time as improvementFactor
FROM justfoo L JOIN justRead R
ON L.n==R.n
;
""", True)

,n,improvementFactor
0,100,3218.111111
1,600,2423.916667
2,1100,2010.887290
3,1600,2084.491057
4,2100,1585.592523
5,2600,2086.262069
6,3100,1952.346405
7,3600,1643.215976
8,4100,1750.643464
9,4600,1774.089888


In [8]:
# draw the improvement factor:
# this graph shows the "slow donw" of the method without I/O over the one with I/O
alt.Chart(improvementFactor).mark_line().encode(
    x='n',
    y=alt.Y('improvementFactor', title='improvement factor')
)

alt.Chart(...)